# Objetivo

El objetivo de este notebook es extraer información relevante de archivos financieros, como son las llamadas trimestrales de ganancias de una empresa. Para ello, se plantean tres tareas principales:

1. **Extracción de entidades:** Se identifican entidades como personas, lugares y empresas, así como sus atributos, como roles o relaciones.
   1. Es una tarea de entity-extraction
2. **Identificación de eventos relevantes:** Se identifican eventos relevantes en el texto y se clasifican como positivos o negativos.
   1. Es una tarea que mezcla summarization con sentiment analysis
3. **Búsqueda de información:** Se construye una pipeline para hacer preguntas al texto.
   1. Es una tarea de Question-answering con Retrieval Augmented Generation

No disponemos de un modelo especializado para este tipo de tareas ni de una gran base de datos con la que pre-entrenar un modelo. Por ello, vamos a utilizar un LLM de gran tamaño, como GPT-3, como punto de partida.

En este caso, no se realizará fine-tuning del modelo, sino que se trabajará con el modelo base. Se experimentará con diferentes prompts y con el acercamiento RAG.

----

**Sumario**

1. Preparar los datos
   1. Cargar el PDF
   2. Dividir en párrafos
   3. Contar número de tokens por párrafo
<br></br>
2. Extraer entidades y relaciones
   1. Preparar el prompt
   2. Correr el prompt por el LLM
   3. Formatear la salida
   4. Comentarios
<br></br>
1. Identificar eventos relevantes
   1. Preparar el prompt
   2. Correr el prompt por el LLM
   3. Formatear la salida
   4. Comentarios
<br></br>
2. Obtener información mediante preguntas
   1. Modelo de embeddings
   2. Base de datos vectorial
   3. Retrieval
   4. Generation
   5. Comentarios

In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "TextEmbeddingAda002", el modelo es "text-embedding-ada-002"
engine = "Davinci003"
model = "text-davinci-003"
openai_api_version = "2023-12-01" 

llm = AzureOpenAI(
    azure_endpoint=openai.api_base, 
    azure_deployment=engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version,
    temperature=0, # Podemos poner la temperatura a 0 si queremos reducir la variabilidad de las respuestas
)

# 1 - Preparar los datos

Nuestro punto de partida es un archivo PDF con el texto de la llamada trimestral. Para utilizar este texto de manera efectiva con nuestro LLM, debemos prepararlo para que se ajuste a la longitud de contexto del modelo. Para ello, seguiremos los siguientes pasos principales:

1. **Cargar el PDF.**

2. **Dividir en párrafos** Este paso nos ayuda a gestionar el texto en trozos más pequeños y digeribles.

3. **Contar tokens por párrafo.** Esta información es crucial para generar prompts que se ajusten a la longitud de contexto del modelo.

## 1.1 - Cargar el PDF

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
import re

# Cargamos el documento
pdf_loader = PyMuPDFLoader("data/growth_point_properties.pdf")
pdf_docs = pdf_loader.load()

## 1.2 - Dividir en párrafos

En este caso voy a utilizar un método propio para dividir el texto en vez de utilizar un textsplitter de LangChain. La razón es que el texto parece estar mal formateado y un splitter normal no funcionaria como yo quiero.

**Para identificar párrafos, usamos una expresion linear que divide el texto cuando se encuentra un salto de línea `\n` seguido de una palabra mayúscula.**

Una mejor alternativa, sobretodo si vamos a tratar con múltiples documentos, sería preformattear el texto o buscar fuentes alternativas del mismo que si esten bien formateadas. ein embargo, vamos a considerar este ejercicio como una "Prueba de Concepto" y por ello, vamos a seguir un acercamiento más simple, trabajando directamente sobre el documento proporcionado.

In [ ]:
def extract_paragraphs_from_pdf(pdf_docs):
    paragraphs = []

    # Iteramos por cada página del PDF
    for pdf_page in pdf_docs:
        page_text = pdf_page.page_content

        # Usamos una expresión regular para identificar parrafos como aquellas lineas que empiezan por "\n" y luego una letra mayúscula
        page_paragraphs = re.split(r'\n(?=[A-Z])', page_text)

        # Eliminamos parrafos vacios
        page_paragraphs = [p.strip() for p in page_paragraphs if p.strip()]

        # Añadimos el parrafo a la lista
        paragraphs.extend(page_paragraphs)

    return paragraphs

# Dividimos el PDF en párrafos
paragraphs = extract_paragraphs_from_pdf(pdf_docs)

# Ahora "paragraphs" es una lista de strings, donde cada string es un párrafo del PDF (aproximadamente). Por ello, eliminamos los saltos de linea "ignorados"
cleaned_paragraphs = [paragraph.replace("\n", "") for paragraph in paragraphs]

# Ahora añadimos un salto de pagina al inicio de cada párrafo para cuando utilicemos esta informacion en los prompts
cleaned_paragraphs = ["\n" + paragraph for paragraph in paragraphs]

In [ ]:
cleaned_paragraphs

## 1.3 - Contar el número de tokens por párrafo

Nuestro objetivo es pasar bloques de texto al LLM para que identifique entidades. Sin embargo, dentro del texto que vamos a pasar al LLM en forma de prompt va a haber información sobre como queremos que se extraigan. Es por ello que tenemos que medir bien los tamaño para que encajen dentro de la ventana de contexto.

**Si ajustamos bien el texto, podemos reducir el número de llamadas que hacemos al modelo, y por tanto reducir la latencia de nuestro sistema.**

Además saber cuantos tokens hay en cada párrafo nos permite saber cuanto cuesta (aproximadamente) cada llamada al modelo ya que en casos como el de GPT-3, el coste de las llamadas depende del número de tokens utilizados.

| Nombre del encoding     | Modelos OpenAI                                       |
|-------------------------|-----------------------------------------------------|
| `cl100k_base`           | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`  |
| `p50k_base`             | Codex models, `text-davinci-002`, `text-davinci-003`|
| `r50k_base` (or `gpt2`) | GPT-3 models like `davinci`                         |

Podemos saber cual es el encoding de un modelo llamando al metodo `tiktoken.encoding_for_model()`:

```python
encoding = tiktoken.encoding_for_model('text-davinci-003')
```

[Ejemplos sobre como contar tokens con `tiktoken`](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [ ]:
import tiktoken
import pandas as pd

def count_tokens(encoding, text):
    return len(encoding.encode(text))

encoding = tiktoken.encoding_for_model(model) # "text-davinci-003"

# Count tokens for each paragraph and store the results in a list of dictionaries
token_counts = []
for paragraph in cleaned_paragraphs:
    num_tokens = count_tokens(encoding, paragraph)
    token_counts.append({'Paragraph': paragraph, 'Token Count': num_tokens})

# Create a Pandas DataFrame from the list of token counts
df = pd.DataFrame(token_counts)
print(f"Número total de tokens en el archivo: {df['Token Count'].sum()}")

df

# 2 - Extraer entidades y relaciones

En esta tarea, combinamos las tareas NLP de "entity-extraction" y "relation-extraction" mediante un enfoque generativo. 

* **Entity-extraction** (extracción de entidades). Se refiere a la identificación y clasificación de entidades mencionadas en el texto, como nombres de personas, lugares , organizaciones, etc.
* **Relation-extraction** (extracción de relaciones). Implica identificar y clasificar las relaciones semánticas entre entidades previamente extraidas. Por ejemplo, en una oración como "Bill gates es fundador de Microsoft", identificariamos la relación "fundador" entre las entidades "Bill Gates" y "Microsoft". Otras posibles relaciones que se pueden extraer indirectamente son: "Bill Gates es emprendedor", "Bill Gates es empresario", etc.

Para lograrlo, utilizaremos [GPT-3](https://platform.openai.com/docs/models/gpt-3-5) con Langchain, empleando una [LLMChain](https://docs.langchain.com/docs/components/chains/llm-chain). Como ya hemos visto, la `LLMChain`  se compone de un PromptTemplate, un modelo y opcionalmente un formateador del output.

Nuestro objetivo implica identificar los siguientes elementos:

* Entidades, como **individuos** y **empresas**.
* Atributos pertinentes asociados con estas entidades, como **roles** o **relaciones**.

Nos centraremos principalmente en tres tipos de relaciones que existen entre entidades:

1. `<is_a>`: Esta relación es útil para definir la naturaleza de empresas, lugares o activos.
2. `<works_at>`: Esta relación indica el lugar de empleo de una persona.
3. `<has_position>`: Esta relación especifica el rol o posición que ocupa una persona dentro de la empresa con la que está asociada.

**Nota:** Si bien este enfoque podria ampliarse a otras relaciones más complejas, hemos optado por centrarnos en estas tres por claridad y con fines de demonstración.

In [ ]:
entity_relationships_task = "entity_relationships"

## 2.1 - Preparar el prompt

### Prompt base

Hemos dividio el prompt en 3 partes, cada una de ellas se encuentra en un archivo de texto:

* `base_template.txt`. Tiene el texto base que da contexto al modelo sobre lo que le vamos a pedir. Basicamente le indicamos que queremos hacer una tarea de extracción, donde las relaciones que encuentre el modelo en el texto deben ser indicadas en el formato `ENTITY_1 <RELATIONSHIP> ENTITY_2`.
* `pcpa.txt`. Indica el tipo de relaciones que nos interesan, y provee al modelo de explicaciones sobre cada una de ellas, para darle contexto.
* `pcpa_microsoft`. Contiene un ejemplo hecho a mano (one-shot learning) donde a partir de un texto, le indicamos que relaciones deberia extraer del mismo. 

La idea de este prompt es explicar al LLM lo mejor posible la tarea para que pueda extraer correctamente entidades en el formato especificado. Como no hemos hecho ningún tipo de fine-tuning, hay que hacer algo de "prompt engineering".

In [ ]:
from pathlib import Path

# Ruta al archivo base de plantilla para tareas de extracción de entidades
base_template_path = Path(f"./prompts/templates/{entity_relationships_task}/base_template.txt")

# Ruta al archivo que muestra las relaciones que nos interesan extraer
relationships_template_path = Path(f"./prompts/templates/{entity_relationships_task}/pcpa.txt")

# Ejemplo de indicación para las relaciones específicas definidas anteriormente
example_prompt_path = Path(f"./prompts/examples/{entity_relationships_task}/pcpa_microsoft.txt")

# Leer el contenido de la plantilla base desde el archivo
with open(base_template_path, 'r') as prompt_file:
    base_template = prompt_file.read()

# Leer el contenido de la plantilla de relaciones desde el archivo
with open(relationships_template_path, 'r') as prompt_file:
    relationships_template = prompt_file.read()

# Leer el contenido del ejemplo de indicación desde el archivo
with open(example_prompt_path, 'r') as prompt_file:
    example_prompt = prompt_file.read()

# Combinar el texto del ejemplo con el template del tipo de relaciones que nos interesan
prompt_text = relationships_template.format(example=example_prompt)

# Combinar el texto de la plantilla base con el prompt anterior
prompt_text = base_template.format(prompt=prompt_text)

# Contar el número de tokens en el texto del prompt
n_prompt_tokens = count_tokens(encoding, prompt_text)
print(n_prompt_tokens)


In [ ]:
print(prompt_text)

### Preparar chunks de texto a analizar

Con el texto anterior ya tenemos el contexto necesario para el LLM. Ahora lo que tenemos que hacer es preparar los chunks de texto de entrada para el modelo.

En este sentido, es muy importante adecuar de forma correcta el tamaño de los chunks, ya que si nos pasamos de tamaño el modelo nos dará un error (o no generará todo lo necesario). Hay 4 variables a considerar cuando decidimos el tamaño de los chunks:

* **El contexto máximo del modelo** (en numero de tokens). En el caso de GPT-3 es 4096. **Esto incluye tanto entrada como generación.**
* **El número máximo de tokens que damos de margen al modelo para generar.**

Además de estos dos, hay que tener que el prompt base ya tiene un numero de tokens (899) y que `tiktoken` no es deterministico (lo he observado empíricamente pero no he investigado la razón). Por ello el número **máximo de tokens del chunk** se puede estimar con la siguiente fórmula:


```python
max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens
max_tokens_per_chunk = 4097 - 1000 - 50 - 899
max_tokens_per_chunk = 2148
```

**Cómo máximo vamos a permitir chunks de 2147 tokens**. Además, como nuestro objetivo es identificar entidades, **vamos a evitar cortar párrafos por la mitad** ya que podria afectar a las entidades o las relaciones.

In [ ]:
MODEL_CONTEXT_LENGTH = 4097
MAX_GENERATION_LENGTH = 1000
EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY = 50 # Not sure why, but I have run the same process multiple times and I have seen different tokenizations, need to double check this
llm.max_tokens = MAX_GENERATION_LENGTH

# Número máximo de tokens por chunk
max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens

print(f"Número máximo de tokens por chunk: {max_tokens_per_chunk}")

def generate_chunks(dataframe, max_tokens_per_chunk):
    # Inicializar variables
    chunks = []
    current_chunk = []
    current_token_count = 0

    # Iterar a través de las filas del DataFrame, que son parrafos del texto
    for index, row in dataframe.iterrows():
        paragraph = row['Paragraph']
        num_tokens = row['Token Count']

        # Si agregar el párrafo actual al fragmento actual no excede el límite
        if current_token_count + num_tokens <= max_tokens_per_chunk:
            current_chunk.append(paragraph)
            current_token_count += num_tokens
        else:
            # Agregar el parrafo actual a la lista de chunks
            if current_chunk:
                chunks.append("".join(current_chunk))
            # Comenzar un nuevo chunk con el párrafo actual
            current_chunk = [paragraph]
            current_token_count = num_tokens

    # Agregar el último chunk (si existe)
    if current_chunk:
        chunks.append("".join(current_chunk))

    return chunks

In [ ]:
chunks = generate_chunks(df, max_tokens_per_chunk)

total_token_count = 0
for i in range(0, len(chunks)):
    token_count = count_tokens(encoding, chunks[i])
    print(f"{i}: {token_count}")
    total_token_count += token_count

print(f"Total: {total_token_count}")

Como podemos ver, se han generado 5 chunks, donde ninguno de ellos excede el máximo de tokens por chunk.

## 2.2 - Correr el prompt por el LLM

In [ ]:
import time

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

def llm_run(chain, query):
    start_time = time.time()  # Registrar el tiempo de inicio

    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    end_time = time.time()  # Registrar el tiempo de finalización
    execution_time = end_time - start_time  # Calcular el tiempo de ejecución en segundos
    print(f'Time taken: {round(execution_time, 2)} seconds')

    return result

# Crear la plantilla del prompt
prompt = PromptTemplate(
    input_variables=["input_text"],
    template=prompt_text + "\n{input_text}",
)

# Crear la cadena LLM simple
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Ejecutar los prompts a través del LLM
er_outputs = []
for i in range(len(chunks)):
    print(f"Chunk {i}")
    output = llm_run(llm_chain, chunks[i])
    er_outputs.append(output)

In [ ]:
er_outputs

Si bien el modelo nos devuelve tripletas de `"Entidad_1" <relacion> "Entidad_2"`, lo hace en forma de bloque de texto. Para poder utilizar dichas relaciones y almacenarlas por ejemplo en una base de datos en grafo como Node4J, tenemos que primero formatearlas correctamente...

In [ ]:
print(er_outputs[0])

## 2.3 - Formatear la salida

Dividimos aquellas lineas de texto que se encuentran en formato `"Entidad_1" <relacion> "Entidad_2"` y las almacenamos en un DataFrame de Pandas. Por supuesto, aquellas que no cumplen con las reglas son ignoradas.

In [ ]:
def split_relationship_text(text):
    # Dividir el texto basándose en "<" y ">" para separar la parte de la relación
    parts = text.split("<")
    
    if len(parts) == 2:
        # Extraer entidades y relación
        entity1 = parts[0].strip()
        relationship, entity2 = parts[1].split(">")
        entity2 = entity2.strip()
        return entity1, relationship, entity2
    else:
        # Manejar de manera elegante la entrada no válida
        print(f"Formato no válido para: {text}")
        return None, None, None

er_output_dfs = []

# Crear DataFrames para cada salida en er_outputs
for output in er_outputs:
    er_output_dfs.append(pd.DataFrame({"Triplet": output.split("\n")}))

# Concatenar los DataFrames
er_output_df = pd.concat(er_output_dfs)

# Eliminar filas vacías y duplicados basados en la columna "Triplet"
er_output_df = er_output_df[er_output_df["Triplet"] != ""]
er_output_df = er_output_df.drop_duplicates(["Triplet"]).reset_index(drop=True).copy()

# Aplicar la función al DataFrame y crear nuevas columnas
er_output_df[['Entidad 1', 'Relación', 'Entidad 2']] = er_output_df['Triplet'].apply(lambda x: pd.Series(split_relationship_text(x)))

In [ ]:
er_output_df

## 2.4 - Comentarios

Como se puede observar, el modelo encuentra dificultades debido a la ausencia de relaciones complejas bien definidas entre entidades, y la falta de una ontología adecuada agrava este problema. En consecuencia, el modelo ocasionalmente asigna incorrectamente relaciones `<works_at>` a empresas y lugares cuando, idealmente, debería utilizar `<is_located_at>` o una relación similar. Esta confusión surge de la limitada flexibilidad del modelo, que se debe a nuestra falta de proporcionar restricciones claras.

Una mejora sencilla es **establecer restricciones que dicten que ciertos tipos de relaciones solo deben existir entre tipos específicos de entidades**. Por ejemplo, podemos marcar como incorrectas las relaciones `<works_at>` que conectan una pareja de entidades que no sean `(Persona, Compañía)`.

La versión compleja seria **proporcionar una ontología que defina claramente los tipos de entidades y las relaciones posibles entre ellas**. Podriamos preparar un proceso iterativo donde las relaciones se van refinando. Tambien estaria bien contar con información extra de las entidades (por ejemplo para saber que realmente Microsoft en cierto contexto es una Compañia, pero en otro puede referise a la marca)

# 3 - Identificar eventos relevantes

Esta tarea es una combinación de **resumen** y **clasificación**:
1. "Resumimos" el texto extrayendo eventos relevantes
2. Clasificamos los eventos como positivos o negativos. 

Para ello, utilizaremos [GPT-3](https://platform.openai.com/docs/models/gpt-3-5) con Langchain, empleando una [LLMChain](https://docs.langchain.com/docs/components/chains/llm-chain). Como ya hemos visto, la `LLMChain`  se compone de un PromptTemplate, un modelo y opcionalmente un formateador del output.

**Nota:** También podríamos probar un enfoque diferente donde dividimos el proceso en dos partes y utilizamos [SimpleSequentialChain](https://python.langchain.com/docs/modules/chains/foundational/sequential_chains), donde la salida de un paso es la entrada al siguiente. **Esto nos permitiría utilizar modelos diferentes para diferentes partes del proceso**. Sería el enfoque que habría elegido si tuviera que trabajar con LLMs menos potentes (también más económicos), sobretodo si contase con más dato para entrenarlos.

* La primera cadena generaría una lista de eventos.
* La segunda cadena tomaría la lista de eventos como entrada y los clasificaría como positivos o negativos.



In [ ]:
business_developments_task = "business_developments"

## 3.1 - Preparar el prompt

### Prompt base

En este caso vamos a trabajar con un template más simple que en la tarea anterior ya que no vamos a proveer de ejemplos (zero-shot learning). En este caso simplemente proveemos de un texto explcativo para el modelo.

**Nota:** Realmente tengo preparado otro con ejemplo, pero haciendo pruebas con el me ha dado peores resultados, por lo que he decido utilizar zero-shot learning mejor que one-shot o few-shot learning

In [ ]:
# Template base para la tarea de identificacion de eventos relevantes
base_template_path = Path(f"./prompts/templates/{business_developments_task}/base_template.txt")

# Template base con un ejemplo (no funciona tan bien)
# base_template_path = Path(f"./prompts/templates/{business_developments_task}/base_template_with_example.txt")

# Leemos el archivo del template
with open(base_template_path, 'r') as prompt_file:
    base_template = prompt_file.read()

prompt_text = base_template

n_prompt_tokens = count_tokens(encoding, prompt_text)

print(n_prompt_tokens)

In [ ]:
print(prompt_text)

### Preparar chunks de texto a analizar

Ya hemos definido previamente la función `generate_chunks()`. En este caso, solo tenemos que utilizarla correctamente:

```python
max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens
max_tokens_per_chunk = 4097 - 1000 - 50 - 220
max_tokens_per_chunk = 2827
```

**Nota:** En este caso me encontré un error algo extraño de OpenAI (`Error in OpenAICallbackHandler.on_llm_end callback: TypeError("unsupported operand type(s) for /: 'NoneType' and 'int'")`), asi que modifique la longitud de los chunks y funciona...

In [ ]:
# En este caso me encontré un error algo extraño de OpenAI: 
# `Error in OpenAICallbackHandler.on_llm_end callback: TypeError("unsupported operand type(s) for /: 'NoneType' and 'int'")` 
# asi que modifique la longitud de los chunks y funciona...
max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens - 50

chunks = generate_chunks(df, max_tokens_per_chunk)

total_token_count = 0
for i in range(0, len(chunks)):
    token_count = count_tokens(encoding, chunks[i])
    print(f"{i}: {token_count}")
    total_token_count += token_count

print(f"Total: {total_token_count}")

## 3.2 - Correr el prompt por el LLM

In [ ]:
# Crear el prompt template
prompt = PromptTemplate(
    input_variables=["input_text"],
    template=prompt_text + "\n{input_text}",
)

# Crear la cadena LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Correr los prompts generados anteriormente por el LLM
bd_outputs = []
for i in range(len(chunks)):
    print(f"Chunk {i}")
    output = llm_run(llm_chain, chunks[i])
    bd_outputs.append(output)

In [ ]:
bd_outputs

## 3.3 - Formatear la salida

Dividimos aquellas lineas de texto que se encuentran en formato `"Evento" | -10 a 10 | "Razón del score"` y las almacenamos en un DataFrame de Pandas. Por supuesto, aquellas que no cumplen con las reglas son ignoradas.

In [ ]:
def split_business_development_text(text):
    parts = text.split('|')
    if len(parts) == 3:
        business_development_summary = parts[0].strip()
        # score = int(parts[1].strip())
        score = parts[1].strip()
        reason_for_score = parts[2].strip()
        return business_development_summary, score, reason_for_score
    else:
        print(f"Invalid format for: {text}")
        return None, None, None

bd_output_dfs = []
for output in bd_outputs:
    bd_output_dfs.append(pd.DataFrame({"RAW Output": output.split("\n")}))

bd_output_df = pd.concat(bd_output_dfs)
bd_output_df = bd_output_df[bd_output_df["RAW Output"] != ""]
bd_output_df = bd_output_df.drop_duplicates(["RAW Output"]).reset_index(drop=True).copy()

# Creamos nuevas columnas
bd_output_df[['Business development', 'Score', 'Explanation']] = bd_output_df['RAW Output'].apply(lambda x: pd.Series(split_business_development_text(x)))
bd_output_df = bd_output_df.dropna()

In [ ]:
bd_output_df

## 3.4 - Comentarios

#### Variabilidad en el formato de los salidas

Es evidente que, incluso aun estableciendo la temperatura del modelo a 0, se generan respuestas en diversos formatos. Por ejemplo:
- Algunas respuestas comienza con listas numeradas
- Las explicaciones de los scores a veces se presentan directamente y otras empizan como `"This is a positive development..."`

Para mitigar esta variabilidad y lograr un formato más consistente, tenemos varios opciones:
- Mejorar los prompts
- Hacer fine-tuning del modelo

En mi opinión, lo que daría mejores resultados es hacer un adecuado fine-tuning del modelo con ejemplos preparados en el formato deseado.

Alternativamente, podríamos considerar volver a ejecutar secciones de texto que no se han formateado según lo previsto. Podríamos usar una configuración de temperatura más alta durante estas ejecuciones y verificar si la salida se alinea con nuestras expectativas.

**Salidas Innecesarias (Desviación de las reglas establecidas en el prompt):**
Como se evidencia, el modelo ocasionalmente genera texto que no sigue las instrucciones que proporcionamos. Afortunadamente, ignoramos estas salidas innecesarias durante el proceso de formateo. Para abordar este problema, deberíamos investigar por qué ocurre esto y explorar mejoras potenciales para evitar tales desviaciones de nuestras pautas en el futuro.

# 4 - Obtener información mediante preguntas

Esta tarea tiene como objetivo poner en práctica lo que hemos visto en la parte de Retrieval Augmented Generation (RAG). Especificamente, es una tarea de pregunta-respuesta con contexto, donde el contexto se almacena en una base de datos vectorial.

Como elementos relevantes de la misma, utilizaremos:
* Base de datos: [Chroma](https://www.trychroma.com/) (en memoria RAM)
* Modelo de embedding: [OpenAI ADA-02](https://platform.openai.com/docs/guides/embeddings)
* Modelo de lenguaje: [OpenAI GPT-3](https://platform.openai.com/docs/models) (`davinci-003`)

En este caso vamos a preparar una aplicación muy simple, por lo que no vamos a introducir memoria, multi-query u otras de las mejoras que hemos discutido en la seccion de RAG avanzado.

## 4.1 - Modelo de embeddings

In [ ]:
import openai
import os
from langchain.embeddings import AzureOpenAIEmbeddings

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "TextEmbeddingAda002", el modelo es "text-embedding-ada-002"
engine = "TextEmbeddingAda002"
openai_api_version = "2023-12-04" 

openai_embeddings_model = AzureOpenAIEmbeddings(azure_endpoint=openai.api_base, azure_deployment=engine, openai_api_key=config.get("OPENAI_API_KEY", ""), openai_api_version=openai.api_version)

## 4.2 - Base de datos vectorial

In [ ]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma

# Ya tenemos el texto dividido en chunks (párrafos en nuestro caso)
chunks = df["Paragraph"].tolist()

# Creamos documentos a partir de los chunks de texto
documents = []
for chunk in chunks:
    doc =  Document(page_content=chunk, metadata={"source": "local"})
    documents.append(doc)

# Guardamos los chunks en una instancia de Chroma que reside en memoria RAM
db = Chroma.from_documents(documents, openai_embeddings_model)

## 4.3 - Retrieval

Ahora escribamos la lógica real de la aplicación. Queremos crear una aplicación sencilla que 
1. Tome una pregunta del usuario
2. Busque documentos relevantes para esa pregunta
3. Pase los documentos recuperados y la pregunta inicial a un LLM
4. Devuelva una respuesta

Primero, necesitamos definir nuestra lógica para buscar en los documentos. LangChain define una interfaz `Retriever` que envuelve un índice que puede devolver Documentos relevantes dado una uery. Su funcionamiento es similar al de `db.similarity_search()`.

El tipo más común de `Retriever` es el `VectorStoreRetriever`, que utiliza las capacidades de búsqueda de similitud de un vector store para facilitar la recuperación.

In [ ]:
# Creamos el retriever, que devuelve los k textos mas similares a la pregunta
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# Corremos una pregunta de ejemplo
retrieved_docs = retriever.invoke("Who are the Chief executives in Growth Point Properties?")

In [ ]:
# Los documentos recuperados
retrieved_docs

Parece que recupera correctamente la información, la cual se encuentra dividida en 2 párrafos (si bien en el texto original realmente es uno) por los problemas del PDF que hemos comentado anteriormente (no se cargan correctamente los saltos de linea).

## 4.4 - Generation

Una vez tenemos los documentos que otorgarán de contexto al LLM, debemos pasarle dicha información dentro de un prompt (junto a la pregunta).

In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "TextEmbeddingAda002", el modelo es "text-embedding-ada-002"
engine = "Davinci003"
model = "text-davinci-003"
openai_api_version = "2023-12-01" 

TEMPERATURE = 0 # low temperature to avoid GPT's "imagination"

llm = AzureOpenAI(
    azure_endpoint=openai.api_base, 
    azure_deployment=engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version,
    temperature=TEMPERATURE,
)

In [ ]:
retrieved_docs

In [ ]:
# Plantilla especifica para Question-Answering
custom_rag_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

{context}

Question: {question}

Answer:"""

custom_rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=custom_rag_template,
)

# Crear la cadena LLM simple
qa_chain = LLMChain(llm=llm, prompt=custom_rag_prompt)

In [ ]:
def qa_run(chain, query):
    start_time = time.time()  # Registrar el tiempo de inicio

    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    end_time = time.time()  # Registrar el tiempo de finalización
    execution_time = end_time - start_time  # Calcular el tiempo de ejecución en segundos
    print(f'Time taken: {round(execution_time, 2)} seconds')

    return result

# El contexto no es mas que la concatenación de los documentos recuperados
context = ""
for i in range (0, len(retrieved_docs)):
    doc = retrieved_docs[i]
    context += f"\nDocument #{i}:"
    context += doc.page_content

# La pregunta
question = "Who are the Chief executives in Growth Point Properties?"

# La query resultante que mandamos al modelo
query = {
    'context': context,
    'question': question
}

In [ ]:
qa_run(qa_chain, query)

## 4.5 - Comentarios

Podemos incrementar la complejidad de la aplicación añadiendo funcionalidades tales como:
* [Streaming](https://python.langchain.com/docs/use_cases/question_answering/streaming). En vez de devolver la respuesta de golpe una vez ha sido generada, podemos ir devolviendola token a token según se genera (similar al comportamiento de Chat-GPT).
  
* [Memoria](https://python.langchain.com/docs/use_cases/question_answering/chat_history). Este sistema está pensado para interacciones puntuales sobre el modelo. No permite una conversación continuada. Para ello, tendriamos que implementar un historial del chat (i.e., una memoria de corto plazo).
  
* [Fuentes](https://python.langchain.com/docs/use_cases/question_answering/sources). Si bien para este ejemplo particular no tiene mucho sentido ya que solo hay un documento sobre el cual estamos haciendo preguntas. Podria darse el caso de que queremos extender la funcionalidad de la aplicacion para analizar un documento de una compañia, sino múltiples documentos, en tal caso, estaria bien que el modelo nos indicase cual es el documento (y que parrafos del mismo) ha utilizado para generar su respuesta.

A su vez, **si los párrafos fueran muy largos, deberiamos hacer estimaciones del número de tokens para poder ajustar lo mejor posible las queries** (similar a las secciones 2 y 3 de este proyecto)

# Siguiente

En la siguiente y última lección, comentaremos ciertos conceptos avanzados como son el despliegue de aplicaciones LangChain, la monitorización o el LangChain Expression Language (LCEL)